In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../../")

from src.chunk.chunk import ChunkStrat
from pathlib import Path

from src.cluster.cluster import ClusterStrategy
from src.cluster.lmp.cluster_v4 import generate_clusters
from src.chunk.chunk import chunk_repo, ChunkStrat

import ell
ell.init(store="logdir")

repo_name = "ell"
repo_path = Path("src/cluster/repos") / repo_name

chunks = chunk_repo(repo_path, ChunkStrat.VANILLA)
chunks_summarized = chunk_repo(repo_path, ChunkStrat.SUMMARY)
chunks_random = chunk_repo(repo_path, ChunkStrat.RANDOM)

chunk_strat = ClusterStrategy(chunks, 
                              cluster_op=generate_clusters)
summarized_strat = ClusterStrategy(chunks_summarized,
                                   cluster_op=generate_clusters)
random_strat = ClusterStrategy(chunks_random,
                                 cluster_op=generate_clusters)

c:\Users\jpeng\Documents\projects\codesearch-backend\.venv\lib\site-packages\starlette\config.py:66: UserWarning: Config file '.env' not found.
  warnings.warn(f"Config file '{env_file}' not found.")


Saving chunks to file:  C:\Users\jpeng\AppData\Local\Temp\index\ell
[Chunker]: 212 chunks used
Saving chunks to file:  C:\Users\jpeng\AppData\Local\Temp\index\ell
[Chunker]: 212 chunks used
Summary tokens: 8400,         Code tokens: 60918,         Ratio: 0.13789027873534915
Saving chunks to file:  C:\Users\jpeng\AppData\Local\Temp\index\ell
[Chunker]: 212 chunks used


In [8]:
from src.llm.evals.closeness import measure_closeness_bytes, measure_closeness_chunks

# Observe the performance of clustering on reduced input sizes
chunk_75 = chunks[:int(len(chunks) * 0.75)]
chunk_50 = chunks[:int(len(chunks) * 0.50)]
chunk_25 = chunks[:int(len(chunks) * 0.25)]

cluster_outputs = []
chunk_inputs = [("25%", chunk_25), ("50%", chunk_50), ("75%", chunk_75)]
for c_name, chunks_i in chunk_inputs:
    print(f"[{c_name}]: size: ", sum([len(chunk.get_content()) for chunk in chunks_i]))

    chunk_strat = ClusterStrategy(chunks_i, 
                              cluster_op=generate_clusters)
    clusters = chunk_strat.run(iters=2)

    cluster_outputs.append(clusters)

    bytes_score = sum([measure_closeness_bytes(cluster, chunks_i) for cluster in clusters]) / len(clusters)
    chunks_score = sum([measure_closeness_chunks(cluster, chunks_i) for cluster in clusters]) / len(clusters)
    print("Bytes score: ", bytes_score)
    print("Chunks score: ", chunks_score)


[25%]: size:  74087
[ELL] use_cache for generate_clusters_raw: False
[ELL] use_cache for identify_key_chunks: False
[ELL] use_cache for format_clusters: False
[ELL] use_cache for generate_clusters_raw: False
[ELL] use_cache for identify_key_chunks: False
[ELL] use_cache for format_clusters: False
Chunk Name: cem.py::6 hallucinated, skipping...
Chunk Name: cpbo.py::6 hallucinated, skipping...
Chunk Name: configurator.py::2 hallucinated, skipping...
Chunk Name: lmp\_complex.py::2 hallucinated, skipping...
Chunk Name: lmp\_complex.py::3 hallucinated, skipping...
Chunk Name: lmp\_complex.py::4 hallucinated, skipping...
Chunk Name: lmp\_complex.py::5 hallucinated, skipping...
Chunk Name: lmp\_tool.py::3 hallucinated, skipping...
Chunk Name: lmp\_tool.py::1 hallucinated, skipping...
Chunk Name: lmp\_tool.py::2 hallucinated, skipping...
Chunk Name: lmp\_tool.py::3 hallucinated, skipping...
Chunk Name: lmp\_complex.py::2 hallucinated, skipping...
Chunk Name: lmp\_complex.py::3 hallucinated, sk

In [18]:
chunk_25 = chunks[:int(len(chunks) * 0.25)]
cluster_25 = cluster_outputs[0]
for c in cluster_25:
    print(measure_closeness_chunks(c, chunk_25))
    print(c)

Curr chunk:  ell\configurator.py::8
Prev chunk:  build.py::2
Curr chunk:  ell\configurator.py::9
Prev chunk:  ell\configurator.py::8
Score!
Curr chunk:  lmp\_track.py::1
Prev chunk:  ell\configurator.py::9
Score!
Curr chunk:  lmp\_track.py::2
Prev chunk:  lmp\_track.py::1
Score!
Curr chunk:  lmp\_track.py::4
Prev chunk:  lmp\_track.py::2
Score!
Curr chunk:  models\__init__.py::1
Prev chunk:  lmp\_track.py::4
Curr chunk:  models\anthropic.py::1
Prev chunk:  models\__init__.py::1
Score!
Curr chunk:  models\bedrock.py::1
Prev chunk:  models\anthropic.py::1
Score!
Curr chunk:  models\groq.py::1
Prev chunk:  models\bedrock.py::1
Score!
Curr chunk:  models\ollama.py::1
Prev chunk:  models\groq.py::1
Score!
0.8
Build and Testing Automation:
build.py::2
ell\configurator.py::8
ell\configurator.py::9
lmp\_track.py::1
lmp\_track.py::2
lmp\_track.py::4
models\__init__.py::1
models\anthropic.py::1
models\bedrock.py::1
models\groq.py::1
models\ollama.py::1


Curr chunk:  0.1.0\cem.py::2
Prev chunk: 

In [19]:
for c in chunk_25:
    print(c.id)

build.py::2
0.1.0\autostreamprevention.py::1
0.1.0\cem.py::1
0.1.0\cem.py::2
0.1.0\cem.py::3
0.1.0\cem.py::4
0.1.0\cem.py::5
0.1.0\cem.py::6
0.1.0\context_versioning.py::1
0.1.0\cpbo.py::1
0.1.0\cpbo.py::2
0.1.0\cpbo.py::3
0.1.0\cpbo.py::4
0.1.0\cpbo.py::5
0.1.0\cpbo.py::6
0.1.0\cpbo.py::7
0.1.0\metapromptingtorch.py::1
0.1.0\mypytest.py::1
0.1.0\test.py::1
src\conf.py::1
src\conf.py::2
ell\__init__.py::1
ell\__version__.py::1
ell\configurator.py::1
ell\configurator.py::2
ell\configurator.py::3
ell\configurator.py::4
ell\configurator.py::5
ell\configurator.py::6
ell\configurator.py::7
ell\configurator.py::8
ell\configurator.py::9
lmp\__init__.py::1
lmp\_track.py::1
lmp\_track.py::2
lmp\_track.py::3
lmp\_track.py::4
lmp\complex.py::1
lmp\complex.py::2
lmp\complex.py::3
lmp\complex.py::4
lmp\complex.py::5
lmp\simple.py::1
lmp\simple.py::2
lmp\tool.py::1
lmp\tool.py::2
lmp\tool.py::3
lmp\tool.py::4
models\__init__.py::1
models\anthropic.py::1
models\bedrock.py::1
models\groq.py::1
models\

In [20]:
test_cluster = cluster_25[-2]
measure_closeness_chunks(test_cluster, chunk_25)

Curr chunk:  models\anthropic.py::1
Prev chunk:  models\__init__.py::1
Score!
Curr chunk:  models\bedrock.py::1
Prev chunk:  models\anthropic.py::1
Score!
Curr chunk:  models\groq.py::1
Prev chunk:  models\bedrock.py::1
Score!
Curr chunk:  models\ollama.py::1
Prev chunk:  models\groq.py::1
Score!


0.2857142857142857